In [1]:
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Define the directory containing the text file and the persistent directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "Libros", "odyssey.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db")


In [2]:
# Check if the Chroma vector store already exists
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    # Ensure the text file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"The file {file_path} does not exist. Please check the path."
        )

    # Read the text content from the file
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split the document into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)

    # Display information about the split documents
    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}\n")

    # Create embeddings
    print("\n--- Creating embeddings ---")
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )  # Update to a valid embedding model if needed
    print("\n--- Finished creating embeddings ---")

    # Create the vector store and persist it automatically
    print("\n--- Creating vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- Finished creating vector store ---")

else:
    print("Vector store already exists. No need to initialize.")


Vector store already exists. No need to initialize.


In [3]:
# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

In [5]:
# Define the user's question
query = "Who is Odysseus' wife?"

In [6]:
# Retrieve relevant documents based on the query
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.3},
)
relevant_docs = retriever.invoke(query)

In [7]:
# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
“Happy Ulysses, son of Laertes,” replied the ghost of Agamemnon, “you
are indeed blessed in the possession of a wife endowed with such rare
excellence of understanding, and so faithful to her wedded lord as
Penelope the daughter of Icarius. The fame, therefore, of her virtue
shall never die, and the immortals shall compose a song that shall be
welcome to all mankind in honour of the constancy of Penelope. How far
otherwise was the wickedness of the daughter of Tyndareus who killed
her lawful husband; her song shall be hateful among men, for she has
brought disgrace on all womankind even on the good ones.”

Source: /media/g/AA89-0893/LangChain_Curso/04_RAGS/Libros/odyssey.txt

Document 2:
Then Ulysses answered, “Madam, wife of Ulysses, you need not defer your
tournament, for Ulysses will return ere ever they can string the bow,
handle it how they will, and send their arrows through the iron.”

To this Penelope said, “As long, sir, as you will sit 